<a href="https://colab.research.google.com/github/seungyounglim/LittleFox_Investment/blob/master/2020_08_17_%ED%95%9C%EA%B5%AD%EA%B1%B0%EB%9E%98%EC%86%8C_%EC%A3%BC%EA%B0%80%EC%A0%95%EB%B3%B4_%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국거래소 기업/시세 공시 정보 제공 서비스
"실시간시세_영문" XML 사용해서 필요한 정보 가져오기   

관련 링크: https://kasp.krx.co.kr/contents/02/02010000/ASP02010000.jsp

In [1]:
import collections
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

DungRak_map = {"1":"상한","2":"상승","3":"보합","4":"하한","5":"보합"}
mapping = {"JongName":"종목명","CurJuka":"현재가","Debi":"전일대비","DungRak":"등락","PrevJuka":"전일종가","Volume":"거래량","Money":"거래대금","StartJuka":"시가","HighJuka":"고가","LowJuka":"저가","High52":"52주최고","Low52":"52주최저","UpJuka":"상한가","DownJuka":"하한가","Per":"PER","Amount":"상장주식수","FaceJuka":"액면가"}
daily_mapping = {"day_Date":"일자","day_EndPrice":"종가","day_getDebi":"전일대비","day_Dungrak":"전일대비(등락)","day_Start":"시가","day_High":"고가","day_Low":"저가","day_Volume":"거래량","day_getAmount":"거래대금"}
ask_mapping = {"member_memdoMem":"매도상위 증권사","member_memdoVol":"매도 거래량", "member_memsoMem":"매수상위 증권사", "member_mesuoVol":"매수 거래량"}


In [2]:
STOCK_CODE = "005380" ## 관심 종목 입력
 
url="http://asp1.krx.co.kr/servlet/krx.asp.XMLSiseEng?code={}".format(STOCK_CODE)
result=urlopen(url).read()
xmlsoup=BeautifulSoup(result,"lxml-xml")

## 1. 실시간 주가정보 가지고 오기

In [3]:
stock = xmlsoup.find("TBL_StockInfo")

df_data = collections.OrderedDict()
for k, v in stock.attrs.items():
  if k == "DungRak": v = DungRak_map[v]
  df_data[mapping[k]] = v
stock_df=pd.DataFrame(df_data, index=[0])
stock_df=stock_df.applymap(lambda x: x.replace(",",""))

stock_df

,종목명,현재가,등락,전일대비,전일종가,거래량,거래대금,시가,고가,저가,52주최고,52주최저,상한가,하한가,PER,상장주식수,액면가
0,HyundaiMotor,167000,보합,3000,170000,4469158,739685611000,171000,172000,161000,179000,65900,221000,119000,14.77,213668187,5000


## 2. 일별 주가 가지고 오기

In [4]:
daily_stock = xmlsoup.find("TBL_DailyStock")
daily_stock = daily_stock.find_all("DailyStock")

reverse_dict = {v:k for k, v in daily_mapping.items()}
daily_stock_df = collections.OrderedDict({k:[] for k in list(daily_mapping.values())})
for dat in daily_stock:
  for k in daily_stock_df.keys():
    if reverse_dict[k] in dat.attrs:
      v = dat.attrs[reverse_dict[k]]
      if reverse_dict[k] == "day_Dungrak": v = DungRak_map[v]
    else:
      v = "None"
    daily_stock_df[k].append(v)

daily_stock_df=pd.DataFrame(daily_stock_df)

In [5]:
daily_stock_df

,일자,종가,전일대비,전일대비(등락),시가,고가,저가,거래량,거래대금
0,20/08/14,"167,000","3,000",보합,"171,000","172,000","161,000","4,469,158","739,685,611,000"
1,20/08/13,"170,000","2,000",보합,"175,000","177,500","167,000","3,997,431","685,908,473,000"
2,20/08/12,"172,000","7,000",보합,"174,000","178,500","170,500","4,877,139","849,763,235,000"
3,20/08/11,"179,000","9,000",상승,"171,500","180,500","164,500","6,776,406","1,160,211,562,122"
4,20/08/10,"170,000","23,000",상승,"151,500","172,500","151,000","8,460,795","1,380,327,846,500"
5,20/08/07,"147,000","2,500",상승,"145,500","149,500","144,500","3,208,082","471,439,724,500"
6,20/08/06,"144,500","10,500",상승,"136,500","146,000","136,000","4,040,334","573,497,284,275"
7,20/08/05,"134,000","1,500",상승,"133,000","135,500","132,000","1,859,551","248,928,134,000"
8,20/08/04,"132,500","5,500",상승,"129,500","134,000","129,000","2,492,506","328,677,840,500"
9,20/08/03,"127,000",500,상승,"126,500","128,500","123,500","1,868,315","236,001,457,000"


## 3. 매도/매수 상위 증권사 정보

In [6]:
ask_price = xmlsoup.find("TBL_AskPrice")
ask_price = ask_price.find_all("AskPrice")

reverse_dict = {v:k for k, v in ask_mapping.items()}
ask_price_df = collections.OrderedDict({k:[] for k in list(ask_mapping.values())})
for dat in ask_price:
  for k in ask_price_df.keys():
    if reverse_dict[k] in dat.attrs:
      v = dat.attrs[reverse_dict[k]]
    else:
      v = "None"
    ask_price_df[k].append(v)

ask_price_df=pd.DataFrame(ask_price_df)

In [7]:
ask_price_df

,매도상위 증권사,매도 거래량,매수상위 증권사,매수 거래량
0,"MIRAE ASSET DAEWOO CO., LTD.","994,507","MIRAE ASSET DAEWOO CO., LTD.","1,078,227"
1,"Kiwoom.com Securities Co., Ltd.","510,038","Kiwoom.com Securities Co., Ltd.","550,238"
2,"BNK Securities Co., Ltd.","468,239","BNK Securities Co., Ltd.","401,218"
3,"Samsung Securities Co., Ltd.","311,210","Samsung Securities Co., Ltd.","327,011"
4,"Korea Investment %26 Securities Co., Ltd.","292,525","Korea Investment %26 Securities Co., Ltd.","316,843"


## 4. 주요 지수 정보

In [9]:
stockInfo = xmlsoup.find("stockInfo")

# df_data = collections.OrderedDict()
# for k, v in stock.attrs.items():
#   if k == "DungRak": v = DungRak_map[v]
#   df_data[mapping[k]] = v
stock_df=pd.DataFrame(stockInfo.attrs, index=[0])
stock_df=stock_df.applymap(lambda x: x.replace(",",""))

stock_df

,kosdaqJisu,kosdaqJisuBuho,kosdaqJisuDebi,starJisu,starJisuBuho,starJisuDebi,jisu50,jisu50Buho,jisu50Debi,myNowTime,myJangGubun,myPublicPrice,krx100Jisu,krx100buho,krx100Debi,kospiJisu,kospiBuho,kospiDebi,kospi200Jisu,kospi200Buho,kospi200Debi
0,835.03,5,19.74,1328.27,2,10.63,,,,2020/08/17 15:09:07,Closed,,5167.31,5,67.35,2407.49,5,30.04,319.24,5,4.09


# 다른 정보 가지고 오기

In [10]:
URL_LINK = {
 "실시간시세_국문": "http://asp1.krx.co.kr/servlet/krx.asp.XMLSise?code={}" ,
 "실시간시세_영문": "http://asp1.krx.co.kr/servlet/krx.asp.XMLSiseEng?code={}" ,
 "공시정보_국문"    : "http://asp1.krx.co.kr/servlet/krx.asp.DisList4MainServlet?code={}&gubun=K" ,
 "공시정보_영문"    : "http://asp1.krx.co.kr/servlet/krx.asp.DisList4MainServlet?code={}&gubun=E" ,
 "재무종합_국문"    : "http://asp1.krx.co.kr/servlet/krx.asp.XMLJemu?code={}",
 "재무종합_영문"    : "http://asp1.krx.co.kr/servlet/krx.asp.XMLJemuEng?code={}",
 "재무종합2_국문" : "http://asp1.krx.co.kr/servlet/krx.asp.XMLJemu2?code={}",
 "재무종합3_국문" : "http://asp1.krx.co.kr/servlet/krx.asp.XMLJemu3?code={}",
 "텍스트" : "http://asp1.krx.co.kr/servlet/krx.asp.XMLText?code={}"
}

def get_url (STOCK_CODE , TYPE):
  result=urlopen(URL_LINK[TYPE].replace("{}", STOCK_CODE)).read()
  return result

In [11]:
TYPE = "재무종합_영문"
result=get_url(STOCK_CODE, TYPE)
xmlsoup=BeautifulSoup(result,"lxml-xml")
xmlsoup

<?xml version="1.0" encoding="utf-8"?>
<?xml version="1.0" encoding="utf-8" ?><financialTotal querytime="2020-08-17 15:09:08">
<TBL_DaeCha month0="" month1="" month2="" year0="" year1="" year2="">
</TBL_DaeCha>
<TBL_SonIk month0="" month1="" month2="" year0="" year1="" year2="">
</TBL_SonIk>
<TBL_CashFlow month0="" month1="" month2="" year0="" year1="" year2="">
</TBL_CashFlow>
</financialTotal>